In [5]:
import os

## 1. Import Libraries

In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\DELL\anaconda3\envs\chatenv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## 2. Intializing Pinecone Vector Database

In [2]:
PINECONE_API_KEY = '7c262767-4267-4485-b039-72cadcb21d34'
index_name = 'rag-chatbot'

In [3]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(index_name)

## 3. Loading PDFs

In [4]:
# Extract data from pdf

def load_pdfs(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [5]:
extractor = load_pdfs('c:\\Users\\DELL\\OneDrive\\Desktop\\MachineLearning\\Generative AI\\rag-chatbot\\data')

In [6]:
# print(extractor[:10])

## 4. Creating Chunks

In [7]:
# Create text chunks

def tokenizer(extracted_data):
    tokens = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = tokens.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks = tokenizer(extractor)

In [9]:
print(len(text_chunks))

1625


## 5. Embeddings storing to Vector DB

In [10]:
# Download embedding Model

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

In [12]:
from sentence_transformers import SentenceTransformer

# Load the model
model_name_lm = "sentence-transformers/all-MiniLM-L6-v2"
model_lm = SentenceTransformer(model_name_lm)

def get_embeddings(texts):
    # Compute embeddings
    embeddings = model_lm.encode(texts, convert_to_numpy=True)
    return embeddings

# Extract text content
texts = [t.page_content for t in text_chunks]

# Generate embeddings
vectors = get_embeddings(texts)

In [13]:
vectors_new = vectors[:999]
print(len(vectors_new))

999


In [14]:

# Prepare data for indexing
upserts = [(str(i), vectors[i]) for i in range(len(vectors_new))]

# Upsert (insert or update) vectors
index.upsert(vectors=upserts)

docsearch = index

In [15]:
query_result = embeddings.embed_query("Hello World")
print(len(query_result))

384


## 6. Retrieving based on similarity search

In [35]:
#If we already have an index we can load it like this
docsearch = Pinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

AttributeError: type object 'Pinecone' has no attribute 'from_existing_index'

## 7. Setting prompt template

In [16]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [17]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

## 8. Initializing LLM Model

In [3]:
llm = CTransformers(model="c:\\Users\\DELL\\OneDrive\\Desktop\\MachineLearning\\Generative AI\\rag-chatbot\\models\\llama-2-7b-chat.ggmlv3.q2_K.bin",
                        model_type="llama",
                        config={'max_new_tokens':512,
                                'temperature':0.8})

RuntimeError: Failed to create LLM 'llama' from 'c:\Users\DELL\OneDrive\Desktop\MachineLearning\Generative AI\rag-chatbot\models\llama-2-7b-chat.ggmlv3.q2_K.bin'.

## 9. Setting Chains for Retrieval

In [4]:
qa = RetrievalQA.from_chain_type(
        llm=llm, 
        chain_type="stuff", 
        retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
        return_source_documents=True, 
        chain_type_kwargs=chain_type_kwargs
    )

NameError: name 'llm' is not defined

## 10. Response Generation

In [5]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

NameError: name 'qa' is not defined